## ■ cifar10 홈페이지 구현하기

## ※ 이론공부는 여기에서 참고할 것! ※

## ■ 신경망 홈페이지를 구현하기 위한 수업준비_20200312 (게시글 58번)

## ■ 로컬에서 cifar10 이미지 분류하는 홈페이지 구현 전체 코드_20200311 (게시글 57번)

## ■ cifar10 R_shiny full code (10기 포트폴리오 게시글 57번)

In [ ]:
setwd("D:\\yys26")

packages <- c('imager', "shiny", "jpeg", "png", "reticulate", "devtools")

if (length(setdiff(packages, rownames(installed.packages()))) > 0) {
  install.packages(setdiff(packages, rownames(installed.packages())))  
}

if (length(setdiff("keras", rownames(installed.packages()))) > 0) {
  devtools::install_github("rstudio/keras") 
}

require(imager)
require(shiny)
require(jpeg)
require(png)
library(reticulate)
library(keras)

#setwd(tempfile())      
#setwd("/Users/aiden/Desktop/data/cifar10_densenet")

load("envir.RData")
model<<-load_model_hdf5("densenet.h5")

synsets <<- readLines("synset.txt")

server <- shinyServer(function(input, output) {
  ntext <- eventReactive(input$goButton, {
    print(input$url)
    if (input$url == "http://") {
      NULL
    } else {
      tmp_file <- tempfile()
      download.file(input$url, destfile = tmp_file, mode = 'wb')
      tmp_file
    }
  })
  
  output$originImage = renderImage({
    list(src = if (input$tabs == "Upload Image") {
      if (is.null(input$file1)) {
        if (input$goButton == 0 || is.null(ntext())) {
          'dog.jpg'
        } else {
          ntext()
        }
      } else {
        input$file1$datapath
      }
    } else {
      if (input$goButton == 0 || is.null(ntext())) {
        if (is.null(input$file1)) {
          'dog.jpg'
        } else {
          input$file1$datapath
        }
      } else {
        ntext()
      }
    },
    title = "Original Image")
  }, deleteFile = FALSE)
  
  output$res <- renderText({
    src = if (input$tabs == "Upload Image") {
      if (is.null(input$file1)) {
        if (input$goButton == 0 || is.null(ntext())) {
          'dog.jpg'
        } else {
          ntext()
        }
      } else {
        input$file1$datapath
      }
    } else {
      if (input$goButton == 0 || is.null(ntext())) {
        if (is.null(input$file1)) {
          'dog.jpg'
        } else {
          input$file1$datapath
        }
      } else {
        ntext()
      }
    }
    
    img <- load.image(src)
    plot(img)
    img <- image_load(src, target_size = c(32,32))
    img
    x <- image_to_array(img)
    # ensure we have a 4d tensor with single element in the batch dimension,
    x <- array_reshape(x, c(1, dim(x)))
    
    # normalize
    x[,,,1] <- (x[,,,1] - mea1) / sds1
    x[,,,2] <- (x[,,,2] - mea2) / sds2
    x[,,,3] <- (x[,,,3] - mea3) / sds3
    
    # predcit
    preds <- model %>% predict(x)
    
    # output result as string
    max.idx <- order(preds[1,], decreasing = TRUE)[1]
    result <- synsets[max.idx]
    res_str <- ""
    tmp <- strsplit(result[1], " ")[[1]]
    res_str <- paste0(res_str, tmp[2])
    res_str
  })
})




require(imager)
require(shiny)
require(jpeg)
require(png)

ui <- shinyUI(
  fluidPage(
    includeCSS("bootstrap.css"),
    
    pageWithSidebar(
      headerPanel(title = 'Image Classification(cifar10) using DenseNet',
                  windowTitle = 'Image Classification(cifar10) using DenseNet'),
      
      fluidRow(
        column(1),
        column(9,
               tabsetPanel(
                 id = "tabs",
                 tabPanel("Upload Image",
                          fileInput('file1', 'Upload a PNG / JPEG File:')),
                 tabPanel(
                   "Use the URL",
                   textInput("url", "Image URL:", "http://"),
                   actionButton("goButton", "Go!")
                 )
               ),
               h3(titlePanel("DESCRIPTION - CIFAR-10 분류")),
               h3(titlePanel("비행기, 자동차, 새, 사슴, 개, 말, 트럭, 고양이, 개구리, 배"))
               
        ),
        column(2)
      ),
      
      
      mainPanel(
        h3("Image"),
        tags$hr(),
        imageOutput("originImage", height = "auto"),
        tags$hr(),
        h3("What is this?"),
        tags$hr(),
        verbatimTextOutput("res")
      )
      
      
    )))

shinyApp(ui = ui, server = server)

## ■ ui.R과 server.R 만들기

## ■ ui.R

In [ ]:
require(imager)
require(shiny)
require(jpeg)
require(png)

shinyUI(
  fluidPage(
    includeCSS("bootstrap.css"),
    
    pageWithSidebar(
      headerPanel(title = 'Image Classification(cifar10) using DenseNet',
                  windowTitle = 'Image Classification(cifar10) using DenseNet'),
      
      fluidRow(
        column(1),
        column(9,
               tabsetPanel(
                 id = "tabs",
                 tabPanel("Upload Image",
                          fileInput('file1', 'Upload a PNG / JPEG File:')),
                 tabPanel(
                   "Use the URL",
                   textInput("url", "Image URL:", "http://"),
                   actionButton("goButton", "Go!")
                 )
               ),
               h3(titlePanel("DESCRIPTION - CIFAR-10 분류")),
               h3(titlePanel("비행기, 자동차, 새, 사슴, 개, 말, 트럭, 고양이, 개구리, 배"))
               
               ),
        column(2)
      ),
      
      
      mainPanel(
        h3("Image"),
        tags$hr(),
        imageOutput("originImage", height = "auto"),
        tags$hr(),
        h3("What is this?"),
        tags$hr(),
        verbatimTextOutput("res")
      )
      
      
    )))


## ■ server.R

In [ ]:
packages <- c('imager', "shiny", "jpeg", "png", "reticulate", "devtools")

if (length(setdiff(packages, rownames(installed.packages()))) > 0) {
  install.packages(setdiff(packages, rownames(installed.packages())))  
}

if (length(setdiff("keras", rownames(installed.packages()))) > 0) {
  devtools::install_github("rstudio/keras") 
}

require(imager)
require(shiny)
require(jpeg)
require(png)
library(reticulate)
library(keras)

#setwd(tempfile())      
#setwd("/Users/aiden/Desktop/data/cifar10_densenet")

load("envir.RData")
model<<-load_model_hdf5("densenet.h5")

synsets <<- readLines("synset.txt")

shinyServer(function(input, output) {
  ntext <- eventReactive(input$goButton, {
    print(input$url)
    if (input$url == "http://") {
      NULL
    } else {
      tmp_file <- tempfile()
      download.file(input$url, destfile = tmp_file, mode = 'wb')
      tmp_file
    }
  })
  
  output$originImage = renderImage({
    list(src = if (input$tabs == "Upload Image") {
      if (is.null(input$file1)) {
        if (input$goButton == 0 || is.null(ntext())) {
          'dog.jpg'
        } else {
          ntext()
        }
      } else {
        input$file1$datapath
      }
    } else {
      if (input$goButton == 0 || is.null(ntext())) {
        if (is.null(input$file1)) {
          'dog.jpg'
        } else {
          input$file1$datapath
        }
      } else {
        ntext()
      }
    },
    title = "Original Image")
  }, deleteFile = FALSE)
  
  output$res <- renderText({
    src = if (input$tabs == "Upload Image") {
      if (is.null(input$file1)) {
        if (input$goButton == 0 || is.null(ntext())) {
          'dog.jpg'
        } else {
          ntext()
        }
      } else {
        input$file1$datapath
      }
    } else {
      if (input$goButton == 0 || is.null(ntext())) {
        if (is.null(input$file1)) {
          'dog.jpg'
        } else {
          input$file1$datapath
        }
      } else {
        ntext()
      }
    }
    
    img <- load.image(src)
    plot(img)
    img <- image_load(src, target_size = c(32,32))
    img
    x <- image_to_array(img)
    # ensure we have a 4d tensor with single element in the batch dimension,
    x <- array_reshape(x, c(1, dim(x)))
    
    # normalize
    x[,,,1] <- (x[,,,1] - mea1) / sds1
    x[,,,2] <- (x[,,,2] - mea2) / sds2
    x[,,,3] <- (x[,,,3] - mea3) / sds3
    
    # predcit
    preds <- model %>% predict(x)
    
    # output result as string
    max.idx <- order(preds[1,], decreasing = TRUE)[1]
    result <- synsets[max.idx]
    res_str <- ""
    tmp <- strsplit(result[1], " ")[[1]]
    res_str <- paste0(res_str, tmp[2])
    res_str
  })
})




## ■ Tip. 모듈 설치 오류시 해결방법

" R_studio에서 밑에 코드 실행해보기 "

In [ ]:
#https://stackoverflow.com/questions/56815446/keras-tensorflow-installation-r
   
#1) Install ANACONDA. 2019.03 X64 2) in Anaconda Prompt:
     
#conda update -n base -c defaults conda
#pip install --upgrade pillow
#conda install -c conda-forge keras tensorflow
#pip install tensorflow --upgrade
   
install.packages("devtools")
library(devtools)
   
devtools::install_github("rstudio/keras",force=TRUE)
library(keras)
install_keras()
help(keras)
#setwd("d:\\data")
sessionInfo()![image.png](attachment:image.png)

## ■ 홈페이지 만들기!!!

In [ ]:
library(rsconnect)



rsconnect::setAccountInfo(name='lullaby0419', 
                          token='76A6ABEE4913A7103F13E9CB0DBD604D', 
                          secret='보안')





rsconnect::deployApp('d:\\yys26',appName = "imagenet")

## ■ imagenet 홈페이지 구현하기


## ■ 1000 개의 이미지 분류 홈페이지 구현 총정리_20200313

http://cafe.daum.net/oracleoracle/ScyF/64 (10기 포트폴리오 게시글 64번)

## ■ imagenet R_shiny full code

In [ ]:
setwd("D:\\yys26")

packages <- c('imager', "shiny", "jpeg", "png", "reticulate", "devtools")

if (length(setdiff(packages, rownames(installed.packages()))) > 0) {
  install.packages(setdiff(packages, rownames(installed.packages())))  
}

if (length(setdiff("keras", rownames(installed.packages()))) > 0) {
  devtools::install_github("rstudio/keras") 
}

require(imager)
require(shiny)
require(jpeg)
require(png)
library(reticulate)
library(keras)

#setwd(tempfile())      
#setwd("/Users/aiden/Desktop/data/cifar10_densenet")

load("envir.RData")
#model<<-load_model_hdf5("densenet.h5")
model <- application_resnet50(weights = 'imagenet')

# synsets <<- readLines("synset.txt")

server <-  shinyServer(function(input, output) {
  ntext <- eventReactive(input$goButton, {
    print(input$url)
    if (input$url == "http://") {
      NULL
    } else {
      tmp_file <- tempfile()
      download.file(input$url, destfile = tmp_file, mode = 'wb')
      tmp_file
    }
  })
  
  output$originImage = renderImage({
    list(src = if (input$tabs == "Upload Image") {
      if (is.null(input$file1)) {
        if (input$goButton == 0 || is.null(ntext())) {
          'dog.jpg'
        } else {
          ntext()
        }
      } else {
        input$file1$datapath
      }
    } else {
      if (input$goButton == 0 || is.null(ntext())) {
        if (is.null(input$file1)) {
          'dog.jpg'
        } else {
          input$file1$datapath
        }
      } else {
        ntext()
      }
    },
    title = "Original Image")
  }, deleteFile = FALSE)
  
  output$res <- renderText({
    src = if (input$tabs == "Upload Image") {
      if (is.null(input$file1)) {
        if (input$goButton == 0 || is.null(ntext())) {
          'dog.jpg'
        } else {
          ntext()
        }
      } else {
        input$file1$datapath
      }
    } else {
      if (input$goButton == 0 || is.null(ntext())) {
        if (is.null(input$file1)) {
          'dog.jpg'
        } else {
          input$file1$datapath
        }
      } else {
        ntext()
      }
    }
    
    img <- load.image(src)
    plot(img)
    
    #img <- image_load(src, target_size = c(32,32))
    
    img <- image_load(src, target_size = c(224,224))
    img
    
    x <- image_to_array(img)
    
    # ensure we have a 4d tensor with single element in the batch dimension,
    
    # the preprocess the input for prediction using resnet50
    
    x <- array_reshape(x, c(1, dim(x)))
    x <- imagenet_preprocess_input(x)
    
    # make predictions then decode and print them
    
    preds <- model %>% predict(x)
    
    imagenet_decode_predictions(preds, top = 1)[[1]][1,2]
    
    # order(preds[1,], decreasing = TRUE)
    # max.idx <- order(preds[1,], decreasing = TRUE)[1]
    # 
    # result <- synsets[max.idx]
    # res_str <- ""
    # tmp <- strsplit(result[1], " ")[[1]]
    # res_str <- paste0(res_str, tmp[2])
    # res_str
    
  })
})




require(imager)
require(shiny)
require(jpeg)
require(png)

ui <- shinyUI(
  fluidPage(
    includeCSS("bootstrap.css"),
    
    pageWithSidebar(
      headerPanel(title = 'Image classificaiton using DenseNet',
                  windowTitle = 'Image Classification using DenseNet'),
      
      fluidRow(
        column(1),
        column(9,
               tabsetPanel(
                 id = "tabs",
                 tabPanel("Upload Image",
                          fileInput('file1', 'Upload a PNG / JPEG File:')),
                 tabPanel(
                   "Use the URL",
                   textInput("url", "Image URL:", "http://"),
                   actionButton("goButton", "Go!")
                 )
               ),
               h3(titlePanel("DESCRIPTION - Image Classification")),
               h3(titlePanel("Image Classification"))
               
        ),
        column(2)
      ),
      
      
      mainPanel(
        h3("Image"),
        tags$hr(),
        imageOutput("originImage", height = "auto"),
        tags$hr(),
        h3("What is this?"),
        tags$hr(),
        verbatimTextOutput("res")
      )
      
      
    )))

shinyApp(ui = ui, server = server)

## ■ ui.R과 server.R 나누기 

## ■ ui.R

In [ ]:
require(imager)
require(shiny)
require(jpeg)
require(png)

shinyUI(
  fluidPage(
    includeCSS("bootstrap.css"),
    
    pageWithSidebar(
      headerPanel(title = 'Image classificaiton using DenseNet',
                  windowTitle = 'Image Classification using DenseNet'),
      
      fluidRow(
        column(1),
        column(9,
               tabsetPanel(
                 id = "tabs",
                 tabPanel("Upload Image",
                          fileInput('file1', 'Upload a PNG / JPEG File:')),
                 tabPanel(
                   "Use the URL",
                   textInput("url", "Image URL:", "http://"),
                   actionButton("goButton", "Go!")
                 )
               ),
               h3(titlePanel("DESCRIPTION - Image Classification")),
               h3(titlePanel("Image Classification"))
               
        ),
        column(2)
      ),
      
      
      mainPanel(
        h3("Image"),
        tags$hr(),
        imageOutput("originImage", height = "auto"),
        tags$hr(),
        h3("What is this?"),
        tags$hr(),
        verbatimTextOutput("res")
      )
      
      
    )))


## ■ server.R

In [ ]:
packages <- c('imager', "shiny", "jpeg", "png", "reticulate", "devtools")

if (length(setdiff(packages, rownames(installed.packages()))) > 0) {
  install.packages(setdiff(packages, rownames(installed.packages())))  
}

if (length(setdiff("keras", rownames(installed.packages()))) > 0) {
  devtools::install_github("rstudio/keras") 
}

require(imager)
require(shiny)
require(jpeg)
require(png)
library(reticulate)
library(keras)

#setwd(tempfile())      
#setwd("/Users/aiden/Desktop/data/cifar10_densenet")

load("envir.RData")
#model<<-load_model_hdf5("densenet.h5")
model <- application_resnet50(weights = 'imagenet')

synsets <<- readLines("synset.txt")

shinyServer(function(input, output) {
  ntext <- eventReactive(input$goButton, {
    print(input$url)
    if (input$url == "http://") {
      NULL
    } else {
      tmp_file <- tempfile()
      download.file(input$url, destfile = tmp_file, mode = 'wb')
      tmp_file
    }
  })
  
  output$originImage = renderImage({
    list(src = if (input$tabs == "Upload Image") {
      if (is.null(input$file1)) {
        if (input$goButton == 0 || is.null(ntext())) {
          'dog.jpg'
        } else {
          ntext()
        }
      } else {
        input$file1$datapath
      }
    } else {
      if (input$goButton == 0 || is.null(ntext())) {
        if (is.null(input$file1)) {
          'dog.jpg'
        } else {
          input$file1$datapath
        }
      } else {
        ntext()
      }
    },
    title = "Original Image")
  }, deleteFile = FALSE)
  
  output$res <- renderText({
    src = if (input$tabs == "Upload Image") {
      if (is.null(input$file1)) {
        if (input$goButton == 0 || is.null(ntext())) {
          'dog.jpg'
        } else {
          ntext()
        }
      } else {
        input$file1$datapath
      }
    } else {
      if (input$goButton == 0 || is.null(ntext())) {
        if (is.null(input$file1)) {
          'dog.jpg'
        } else {
          input$file1$datapath
        }
      } else {
        ntext()
      }
    }
    
    img <- load.image(src)
    plot(img)
    
    #img <- image_load(src, target_size = c(32,32))
    
    img <- image_load(src, target_size = c(224,224))
    img
    
    x <- image_to_array(img)
    
    # ensure we have a 4d tensor with single element in the batch dimension,
    
    # the preprocess the input for prediction using resnet50
    
     x <- array_reshape(x, c(1, dim(x)))
     x <- imagenet_preprocess_input(x)
     
    # make predictions then decode and print them

     preds <- model %>% predict(x)
     
     imagenet_decode_predictions(preds, top = 1)[[1]][1,2]
     
     # order(preds[1,], decreasing = TRUE)
     # max.idx <- order(preds[1,], decreasing = TRUE)[1]
     # 
     # result <- synsets[max.idx]
     # res_str <- ""
     # tmp <- strsplit(result[1], " ")[[1]]
     # res_str <- paste0(res_str, tmp[2])
     # res_str
     
  })
})


### ※ imagenet_decode_predictions(preds, top = 1)[[1]][1,2]

sysnet으로 결과를 출력하는 것이 아니라 위의 이 코드부분으로 결과값을 출력해준다.

## ■ 홈페이지 만들기!!!

### yys27.zip을 다운로드받은후 하면 됨.

In [ ]:
library(rsconnect)



rsconnect::setAccountInfo(name='lullaby0419', 
                          token='76A6ABEE4913A7103F13E9CB0DBD604D', 
                          secret='보안')





rsconnect::deployApp('d:\\yys27',appName = "imagenet")